## Neural Networks
Neural networks can be constructed using the torch.nn package, and `nn` depends on `autogrd` to define models and differentiate them. An `nn.module` contains layers and a `forward` method that defines how the input data passes through the network.

### Procedure to train a neural network
- Define the model, showing learnable parameters or weights.
- Iterate over dataset inputs
- Process inputs through the model to get outputs.
- Calculate the loss using a loss function, how far the output is from the expected result.
- Backpropagate the loss to compute gradients over the network's parameters.
- Update the parameters using an optimizer, which adjusts the weights based on the gradients. A typical simple update rule is `w = w - learning_rate * gradient`.

### Define the model

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, input):
        # Convolution layer C1: 1 input image channel, 6 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a Tensor with size (N, 6, 28, 28), where N is the size of the batch
        c1 = F.relu(self.conv1(input))
        # Subsampling layer S2: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 6, 14, 14) Tensor
        s2 = F.max_pool2d(c1, (2, 2))
        # Convolution layer C3: 6 input channels, 16 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a (N, 16, 10, 10) Tensor
        c3 = F.relu(self.conv2(s2))
        # Subsampling layer S4: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 16, 5, 5) Tensor
        s4 = F.max_pool2d(c3, 2)
        # Flatten operation: purely functional, outputs a (N, 400) Tensor
        s4 = torch.flatten(s4, 1)
        # Fully connected layer F5: (N, 400) Tensor input,
        # and outputs a (N, 120) Tensor, it uses RELU activation function
        f5 = F.relu(self.fc1(s4))
        # Fully connected layer F6: (N, 120) Tensor input,
        # and outputs a (N, 84) Tensor, it uses RELU activation function
        f6 = F.relu(self.fc2(f5))
        # Gaussian layer OUTPUT: (N, 84) Tensor input, and
        # outputs a (N, 10) Tensor
        output = self.fc3(f6)
        return output


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


As you can see, you just have to define the forward function and the `backward` function is automatically defined by autograd. You can use any Tensor operation in the forward function.

The learnable parameters of a model can be accessed using the `.parameters()` method.

In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


Let's try a random 32x32 input.

In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0517,  0.1201,  0.0959,  0.0759, -0.0453,  0.1409,  0.1188, -0.0725,
          0.1256,  0.0592]], grad_fn=<AddmmBackward0>)


Then let's zero the gradient buffers of all parameters and backprops with random gradients:

In [ ]:
net.zero_grad()
out.backward(torch.randn(1, 10))

### Loss function

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different loss functions under the nn package . A simple loss is: nn.MSELoss which computes the mean-squared error between the output and the target.

For example:

In [4]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6092, grad_fn=<MseLossBackward0>)


Now, if you follow loss in the backward direction, using its .grad_fn attribute, you will see a graph of computations that looks like this:

In [5]:
print(loss.grad_fn)  # None, because loss is a scalar

So, when we call loss.backward(), the whole graph is differentiated w.r.t. the neural net parameters, and all Tensors in the graph that have requires_grad=True will have their .grad Tensor accumulated with the gradient.

In [6]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

### Backprop

To backpropagate the error all we have to do is to loss.backward(). You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

Now we shall call loss.backward(), and have a look at conv1’s bias gradients before and after the backward.

In [7]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0019, -0.0109,  0.0101, -0.0192,  0.0091, -0.0075])


The neural network package contains various modules and loss functions that form the building blocks of deep neural networks. A full list with documentation is [here](https://pytorch.org/docs/nn).



### Update the weights

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):

```python
weight = weight - learning_rate * gradient
```

We can implement this using simple Python code:

In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: torch.optim that implements all these methods. Using it is very simple:

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers to avoid gradient accumulation
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

### Recap of the learned classes

- `torch.Tensor`: A multi-dimensional array with support for autograd like `backward()`, and holds the gradient w.r.t. the tensor.
- `torch.nn.Module`: The base class for all neural network modules, containing layers and a forward method.Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc
- `torch.nn.Parameter`: A kind of Tensor that is automatically registered as a parameter when assigned as an attribute to a Module. It is used to define learnable parameters in a model.
- `autograd.Function`: implements forward and backward definitions of an autograd operation. Every Tensor operation creates at least a single Function object that stores the operation's history, allowing for automatic differentiation.